<p style="text-align: center; color: black; font-size: 48px;"><strong>Challenge : prédire le sentiments des tweets (positif ou négatif)</strong></p>

In [31]:
import pandas as pd
import numpy as np
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')
#nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/solenedegrutere/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/solenedegrutere/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/solenedegrutere/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/solenedegrutere/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

<p style="color: black;"><strong>1. Importe ce dataset de tweets dans un DataFrame.</strong></p>


In [32]:
import pandas as pd
df = pd.read_csv('nlp/data/train.csv')
df.head()

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative


<p style="color: black;"><strong>2. Conserve uniquement les tweets positifs et négatifs (donc tu exclus les neutrals). Quel est le pourcentage de tweets positifs/négatifs ?</strong></p>

In [33]:
df_filtered = df[df['sentiment'].isin(['positive', 'negative'])]
positive_percentage = (df_filtered['sentiment'] == 'positive').mean() * 100
negative_percentage = (df_filtered['sentiment'] == 'negative').mean() * 100

print("Pourcentage de tweets positifs :",positive_percentage, "%")
print("Pourcentage de tweets négatifs :",negative_percentage, "%")


Pourcentage de tweets positifs : 52.44759518425717 %
Pourcentage de tweets négatifs : 47.55240481574283 %


<p style="color: black;"><strong>3. Crée une fonction que tu appelleras clean qui prend pour paramètre une phrase (donc un texte str) et qui retourne un texte (str) de tokens après application d'un stemmer ou d'un lemmatizer, séparés par des espaces.</strong></p>

<p style="color: black;"><strong>4. Récupère la liste des stopwords anglais depuis NLTK, et copie-la dans une liste stopwordsenglish. Complète ta fonction clean pour qu'elle supprime la ponctuation et les stopwords.</strong></p>

In [34]:
lemmatizer = WordNetLemmatizer()
stopwordsenglish = stopwords.words('english')

# Fonction pour mapper les tags NLTK vers WordNet
def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN  # Par défaut

def clean(text):
    text = text.lower()
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in string.punctuation]
    tokens = [word for word in tokens if word not in stopwordsenglish]

    tagged_tokens = pos_tag(tokens)
    lemmatized = [
        lemmatizer.lemmatize(word, get_wordnet_pos(pos))
        for word, pos in tagged_tokens
    ]
    
    return ' '.join(lemmatized)

# Test rapide
print("✅ Test clean() :", clean("You are better when I am well."))


✅ Test clean() : well well


<p style="color: black;"><strong>5. Applique cette fonction clean à la colonne text de ton DataFrame. Stocke le résultat dans une nouvelle colonne clean du DataFrame. (Le traitement peut durer 2 ou 3 minutes)</strong></p>

In [35]:
df_filtered ['clean'] = df_filtered ['text'].apply(clean)
df_filtered 

/var/folders/f0/l0pmr7js5m7fqgqkychtm5cw0000gn/T/ipykernel_23535/944896778.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered ['clean'] = df_filtered ['text'].apply(clean)


,textID,text,selected_text,sentiment,clean
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,sooo sad miss san diego
2,088c60f138,my boss is bullying me...,bullying me,negative,bos bullying ...
3,9642c003ef,what interview! leave me alone,leave me alone,negative,interview leave alone
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative,son put release already buy
6,6e0c6d75b1,2am feedings for the baby are fun when he is a...,fun,positive,2am feeding baby fun smile coo
...,...,...,...,...,...
27474,b78ec00df5,enjoy ur night,enjoy,positive,enjoy ur night
27475,4eac33d1c0,wish we could come see u on Denver husband l...,d lost,negative,wish could come see u denver husband lose job ...
27476,4f4c4fc327,I`ve wondered about rake to. The client has ...,", don`t force",negative,wonder rake client make clear .net force devs ...
27477,f67aae2310,Yay good for both of you. Enjoy the break - y...,Yay good for both of you.,positive,yay good enjoy break probably need hectic week...


<p style="color: black;"><strong>6. Copie la colonne clean dans une Serie X, et la colonne sentiment dans une Serie y. Applique un train-test split avec la taille du jeu d'entrainement à 0.75 avec le random_state = 32.</strong></p>

In [36]:
X = df_filtered ['clean']
y = df_filtered ['sentiment']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=32)


<p style="color: black;"><strong>7. Applique un CountVectorizer et entraine des modèles de classification.</strong></p>

In [37]:
print("\n🔎 Logistic Regression avec CountVectorizer")
cv = CountVectorizer()
X_train_cv = cv.fit_transform(X_train)
X_test_cv = cv.transform(X_test)

lr_cv = LogisticRegression(max_iter=1000)
lr_cv.fit(X_train_cv, y_train)
y_pred_cv = lr_cv.predict(X_test_cv)
print("Accuracy (CountVectorizer) :", accuracy_score(y_test, y_pred_cv))
print(classification_report(y_test, y_pred_cv))


🔎 Logistic Regression avec CountVectorizer
Accuracy (CountVectorizer) : 0.8670251772182841
              precision    recall  f1-score   support

    negative       0.85      0.87      0.86      1935
    positive       0.88      0.87      0.87      2156

    accuracy                           0.87      4091
   macro avg       0.87      0.87      0.87      4091
weighted avg       0.87      0.87      0.87      4091



<p style="color: black;"><strong>8. Applique un TfidfVectorizer et entraine des modèles de classification.</strong></p>

In [38]:
print("\n🔎 Logistic Regression avec TfidfVectorizer")
tfidf = TfidfVectorizer()
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

lr_tfidf = LogisticRegression(max_iter=1000)
lr_tfidf.fit(X_train_tfidf, y_train)
y_pred_tfidf = lr_tfidf.predict(X_test_tfidf)
print("Accuracy (TfidfVectorizer) :", accuracy_score(y_test, y_pred_tfidf))
print(classification_report(y_test, y_pred_tfidf))


🔎 Logistic Regression avec TfidfVectorizer
Accuracy (TfidfVectorizer) : 0.8704473233928135
              precision    recall  f1-score   support

    negative       0.86      0.87      0.86      1935
    positive       0.88      0.87      0.88      2156

    accuracy                           0.87      4091
   macro avg       0.87      0.87      0.87      4091
weighted avg       0.87      0.87      0.87      4091



<p style="color: black;"><strong>Bonus : maintenant c'est à toi d'améliorer ton modèle :</strong></p>

<p style="color: black;"> - En cherchant des paramètres de modèles : par gridsearch et crossvalidation par exemple ;</p>


In [39]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

# Pipeline vectorisation + classification
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', LogisticRegression(max_iter=1000))
])

# Grille d'hyperparamètres à tester
param_grid = {
    'tfidf__ngram_range': [(1, 1), (1, 2)],  # Unigrammes ou bigrammes
    'tfidf__min_df': [1, 3],
    'tfidf__max_df': [0.9, 1.0],
    'clf__C': [0.1, 1, 10],  # Régularisation
    'clf__penalty': ['l2'],
    'clf__solver': ['liblinear']  # adapté pour petits datasets
}

# GridSearch
grid = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy', n_jobs=-1, verbose=1)
grid.fit(X_train, y_train)

# Résultats
print("✅ Meilleurs paramètres :", grid.best_params_)
best_model = grid.best_estimator_

y_pred = best_model.predict(X_test)
print("✅ Accuracy après GridSearch :", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Fitting 5 folds for each of 24 candidates, totalling 120 fits
✅ Meilleurs paramètres : {'clf__C': 1, 'clf__penalty': 'l2', 'clf__solver': 'liblinear', 'tfidf__max_df': 0.9, 'tfidf__min_df': 3, 'tfidf__ngram_range': (1, 2)}
✅ Accuracy après GridSearch : 0.8687362503055488
              precision    recall  f1-score   support

    negative       0.86      0.87      0.86      1935
    positive       0.88      0.87      0.87      2156

    accuracy                           0.87      4091
   macro avg       0.87      0.87      0.87      4091
weighted avg       0.87      0.87      0.87      4091



<p style="color: black;">- En changeant la préparation du texte : par exemple certaines ponctuations peuvent aider le modèle, le point d'exclamation notamment.</p>


In [42]:
def clean2(text):
    text = text.lower()
    keep = ['!', '?']
    tokens = word_tokenize(text)
    
    # Conserver ponctuation expressive (!, ?) mais supprimer le reste
    tokens = [t for t in tokens if (t in keep or t not in string.punctuation)]
    tokens = [t for t in tokens if t not in stopwordsenglish]
    
    tagged = pos_tag(tokens)
    lemmatized = [lemmatizer.lemmatize(word, get_wordnet_pos(pos)) for word, pos in tagged]
    return ' '.join(lemmatized)


In [43]:
df_filtered ['clean_2'] = df_filtered ['text'].apply(clean2)
df_filtered 

/var/folders/f0/l0pmr7js5m7fqgqkychtm5cw0000gn/T/ipykernel_23535/564796572.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered ['clean_2'] = df_filtered ['text'].apply(clean2)


,textID,text,selected_text,sentiment,clean,clean_2
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,sooo sad miss san diego,sooo sad miss san diego ! ! !
2,088c60f138,my boss is bullying me...,bullying me,negative,bos bullying ...,bos bullying ...
3,9642c003ef,what interview! leave me alone,leave me alone,negative,interview leave alone,interview ! leave alone
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative,son put release already buy,son put release already buy
6,6e0c6d75b1,2am feedings for the baby are fun when he is a...,fun,positive,2am feeding baby fun smile coo,2am feeding baby fun smile coo
...,...,...,...,...,...,...
27474,b78ec00df5,enjoy ur night,enjoy,positive,enjoy ur night,enjoy ur night
27475,4eac33d1c0,wish we could come see u on Denver husband l...,d lost,negative,wish could come see u denver husband lose job ...,wish could come see u denver husband lose job ...
27476,4f4c4fc327,I`ve wondered about rake to. The client has ...,", don`t force",negative,wonder rake client make clear .net force devs ...,wonder rake client make clear .net force devs ...
27477,f67aae2310,Yay good for both of you. Enjoy the break - y...,Yay good for both of you.,positive,yay good enjoy break probably need hectic week...,yay good enjoy break probably need hectic week...


In [44]:
X = df_filtered ['clean2']
y = df_filtered ['sentiment']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=32)

KeyError: 'clean2'

In [45]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

# Pipeline vectorisation + classification
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', LogisticRegression(max_iter=1000))
])

# Grille d'hyperparamètres à tester
param_grid = {
    'tfidf__ngram_range': [(1, 1), (1, 2)],  # Unigrammes ou bigrammes
    'tfidf__min_df': [1, 3],
    'tfidf__max_df': [0.9, 1.0],
    'clf__C': [0.1, 1, 10],  # Régularisation
    'clf__penalty': ['l2'],
    'clf__solver': ['liblinear']  # adapté pour petits datasets
}

# GridSearch
grid = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy', n_jobs=-1, verbose=1)
grid.fit(X_train, y_train)

# Résultats
print("✅ Meilleurs paramètres :", grid.best_params_)
best_model = grid.best_estimator_

y_pred = best_model.predict(X_test)
print("✅ Accuracy après GridSearch :", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Fitting 5 folds for each of 24 candidates, totalling 120 fits
✅ Meilleurs paramètres : {'clf__C': 1, 'clf__penalty': 'l2', 'clf__solver': 'liblinear', 'tfidf__max_df': 0.9, 'tfidf__min_df': 3, 'tfidf__ngram_range': (1, 2)}
✅ Accuracy après GridSearch : 0.8687362503055488
              precision    recall  f1-score   support

    negative       0.86      0.87      0.86      1935
    positive       0.88      0.87      0.87      2156

    accuracy                           0.87      4091
   macro avg       0.87      0.87      0.87      4091
weighted avg       0.87      0.87      0.87      4091

